Data Cleaning

In [1]:
import pandas as pd

# Load the Excel sheet
file_path = 'Small_test_data.csv'  # Replace with your actual file path
sheet_name = 'Sheet2'  # Update if your sheet has a different name
df = pd.read_csv(file_path)

# List of address columns
address_columns = ['billingstreet', 'billingcity', 'billingstate', 'billingpostalcode', 'billingcountry']

# Clean each column by removing extra spaces
for column in address_columns:
    if column in df.columns:
        df[column] = df[column].astype(str).str.strip()  # Convert to string if needed and strip spaces

# Save the cleaned data to a new Excel file
cleaned_file_path = 'cleaned_test_data.csv'
df.to_csv(cleaned_file_path, index=False)

print(f"Data cleaning complete. Cleaned data saved to {cleaned_file_path}")

Data cleaning complete. Cleaned data saved to cleaned_test_data.csv


In [10]:
import pandas as pd
from chromadb import Client
from sentence_transformers import SentenceTransformer

# Load your data
df = pd.read_csv("cleaned_test_data.csv")

# Initialize embedding model and ChromaDB client
model = SentenceTransformer('all-MiniLM-L6-v2')  # Use an appropriate model
client = Client()

# Create or access a collection for address fields
collection = client.get_or_create_collection(name="address_fields_test")

# Embed and insert each field into ChromaDB with field-specific metadata
for _, row in df.iterrows():
    metadata = {
        "street": row["billingstreet"],
        "city": row["billingcity"],
        "state": row["billingstate"],
        "postal_code": row["billingpostalcode"],
        "country": row["billingcountry"]
    }
    
    # Embed each field separately
    for field, value in metadata.items():
        if value:  # Ensure field is not empty
            embedding = model.encode(value).tolist()
            
            # Each record needs a unique ID in ChromaDB
            unique_id = f"{field}_{_}"
            
            # Use the correct ChromaDB syntax to add the embedding with metadata
            collection.add(
                documents=[value],
                metadatas=[metadata],
                ids=[unique_id],
                embeddings=[embedding]
            )

# Query by individual fields
query_value = "500 Clipper Dr"  # For example, querying with city name
query_embedding = model.encode(query_value).tolist()
results = collection.query(query_embeddings=[query_embedding], n_results=1)

# Format the output by fetching the correct field values
# for result in results['documents']:
#     print(results)

    # print(f"Street: {result['metadata'].get('street', 'N/A')}")
    # print(f"City: {result['metadata'].get('city', 'N/A')}")
    # print(f"State: {result['metadata'].get('state', 'N/A')}")
    # print(f"Postal Code: {result['metadata'].get('postal_code', 'N/A')}")
    # print(f"Country: {result['metadata'].get('country', 'N/A')}")
for metadata in results['metadatas']:
    metadata_entry = metadata[0]  # Access the first (and only) item in the nested list
    print(f"Street: {metadata_entry.get('street', 'N/A')}")
    print(f"City: {metadata_entry.get('city', 'N/A')}")
    print(f"State: {metadata_entry.get('state', 'N/A')}")
    print(f"Postal Code: {metadata_entry.get('postal_code', 'N/A')}")
    print(f"Country: {metadata_entry.get('country', 'N/A')}")


Add of existing embedding ID: street_0
Insert of existing embedding ID: street_0
Add of existing embedding ID: city_0
Insert of existing embedding ID: city_0
Add of existing embedding ID: state_0
Insert of existing embedding ID: state_0
Add of existing embedding ID: postal_code_0
Insert of existing embedding ID: postal_code_0
Add of existing embedding ID: country_0
Insert of existing embedding ID: country_0
Add of existing embedding ID: street_1
Insert of existing embedding ID: street_1
Add of existing embedding ID: city_1
Insert of existing embedding ID: city_1
Add of existing embedding ID: state_1
Insert of existing embedding ID: state_1
Add of existing embedding ID: postal_code_1
Insert of existing embedding ID: postal_code_1
Add of existing embedding ID: country_1
Insert of existing embedding ID: country_1
Add of existing embedding ID: street_2
Insert of existing embedding ID: street_2
Add of existing embedding ID: city_2
Insert of existing embedding ID: city_2
Add of existing embe

Street: 500 Clipper Dr
City: Belmont
State: California
Postal Code: 94002-4169
Country: United States


In [17]:
# Query by individual fields
query_value = "Pfizer700 Chesapeake Dr"  # For example, querying with city name
query_embedding = model.encode(query_value).tolist()
results = collection.query(query_embeddings=[query_embedding], n_results=1)

for metadata in results['metadatas']:
    metadata_entry = metadata[0]  # Access the first (and only) item in the nested list
    print(f"Street: {metadata_entry.get('street', 'N/A')}")
    print(f"City: {metadata_entry.get('city', 'N/A')}")
    print(f"State: {metadata_entry.get('state', 'N/A')}")
    print(f"Postal Code: {metadata_entry.get('postal_code', 'N/A')}")
    print(f"Country: {metadata_entry.get('country', 'N/A')}")

Street: 700 Chesapeake Dr
City: Redwood City
State: California
Postal Code: 94063-4731
Country: United States
